In [26]:
# Imports -  These are all you need for the assignment: do not import additional packages
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import requests
from geopy.geocoders import Nominatim
from urllib.request import Request


import patsy
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest


In [27]:
#load Los Angeles Data
df_LosAngeles = pd.read_csv("data/Crime_Data_from_2010_to_Present.csv") 

In [28]:
#Load Median Housing Data
df_HousePrice = pd.read_csv("data/Zip_MedianListingPrice_AllHomes.csv")

#Filter by Los Angeles Only
df_LAHousePrice = df_HousePrice[df_HousePrice['City'] == 'Los Angeles']

In [29]:
#locator hack

def get_geolocator():
    geolocator = Nominatim()

    requester = geolocator.urlopen

    def requester_hack(req, **kwargs):
        req = Request(url=req, headers=geolocator.headers)
        return requester(req, **kwargs)

    geolocator.urlopen = requester_hack

    return geolocator

In [30]:
#Convert location to zip code
location = get_geolocator().reverse("52.509669, 13.376294")
address = location.address
print(location)


Backwerk, Potsdamer Platz, Tiergarten, Mitte, Berlin, 10785, Deutschland


In [31]:
df_LosAngeles

,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,...,Weapon Description,Status Code,Status Description,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,Location
0,1208575,03/14/2013,03/11/2013,1800,12,77th Street,1241,626,INTIMATE PARTNER - SIMPLE ASSAULT,0416 0446 1243 2000,...,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",AO,Adult Other,626.0,NaN,NaN,NaN,6300 BRYNHURST AV,NaN,"(33.9829, -118.3338)"
1,102005556,01/25/2010,01/22/2010,2300,20,Olympic,2071,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,VAN NESS,15TH,"(34.0454, -118.3157)"
2,418,03/19/2013,03/18/2013,2030,18,Southeast,1823,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,200 E 104TH ST,NaN,"(33.942, -118.2717)"
3,101822289,11/11/2010,11/10/2010,1800,18,Southeast,1803,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,88TH,WALL,"(33.9572, -118.2717)"
4,42104479,01/11/2014,01/04/2014,2300,21,Topanga,2133,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),0329,...,NaN,IC,Invest Cont,745.0,NaN,NaN,NaN,7200 CIRRUS WY,NaN,"(34.2009, -118.6369)"
5,120125367,01/08/2013,01/08/2013,1400,1,Central,111,110,CRIMINAL HOMICIDE,1243 2000 1813 1814 2002 0416 0400,...,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",AA,Adult Arrest,110.0,NaN,NaN,NaN,600 N HILL ST,NaN,"(34.0591, -118.2412)"
6,101105609,01/28/2010,01/27/2010,2230,11,Northeast,1125,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,YORK,AVENUE 51,"(34.1211, -118.2048)"
7,101620051,11/11/2010,11/07/2010,1600,16,Foothill,1641,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,EL DORADO,TRUESDALE,"(34.241, -118.3987)"
8,101910498,04/07/2010,04/07/2010,1600,19,Mission,1902,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,GLENOAKS,DRELL,"(34.3147, -118.4589)"
9,120908292,03/29/2013,01/15/2013,800,9,Van Nuys,904,668,"EMBEZZLEMENT, GRAND THEFT ($950.01 & OVER)",0344 1300,...,NaN,IC,Invest Cont,668.0,NaN,NaN,NaN,7200 SEPULVEDA BL,NaN,"(34.2012, -118.4662)"


In [32]:
df_HousePrice

,RegionName,City,State,Metro,CountyName,SizeRank,2010-01,2010-02,2010-03,2010-04,...,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01
0,10025,New York,NY,New York,New York,1,785000.0,730000.0,775000.0,725000.0,...,1497500.0,1467000.0,1400000.0,1332500.0,1325000.0,1297500.0,1400000.0,1495000.0,1575000.0,1500000.0
1,60657,Chicago,IL,Chicago,Cook,2,349900.0,349000.0,339169.5,347950.0,...,499900.0,499000.0,500000.0,485000.0,485000.0,514000.0,500000.0,499999.5,529900.0,515000.0
2,10023,New York,NY,New York,New York,3,NaN,NaN,NaN,NaN,...,2425000.0,2395000.0,2400000.0,2495000.0,2395000.0,2395000.0,2375000.0,2395000.0,2495000.0,2395000.0
3,77494,Cinco Ranch,TX,Houston,Fort Bend,4,285000.0,285000.0,289900.0,289055.0,...,378875.0,380000.0,379700.0,375000.0,375000.0,374900.0,374000.0,369290.0,365000.0,364995.0
4,75070,McKinney,TX,Dallas-Fort Worth,Collin,5,191315.0,194900.0,190995.0,195000.0,...,373940.0,371990.0,368000.0,351138.5,350000.0,361092.0,369900.0,365000.0,368950.0,367990.0
5,60614,Chicago,IL,Chicago,Cook,6,NaN,NaN,NaN,NaN,...,675000.0,674900.0,665000.0,669950.0,660000.0,679000.0,695000.0,749999.0,765000.0,775000.0
6,79936,El Paso,TX,El Paso,El Paso,7,134900.0,130000.0,129900.0,129900.0,...,139925.0,139900.0,140000.0,139900.0,139900.0,139250.0,139000.0,135000.0,139950.0,135000.0
7,77084,Houston,TX,Houston,Harris,8,119900.0,119850.0,121950.0,120000.0,...,185000.0,189900.0,189900.0,189992.5,189990.0,189900.0,184900.0,175000.0,175000.0,179250.0
8,10002,New York,NY,New York,New York,9,NaN,NaN,NaN,NaN,...,1600000.0,1575000.0,1575000.0,1577500.0,1580000.0,1612000.0,1605000.0,1665500.0,1630000.0,1630000.0
9,60640,Chicago,IL,Chicago,Cook,10,NaN,NaN,NaN,NaN,...,265000.0,259900.0,270000.0,265000.0,253950.0,256500.0,269000.0,274950.0,269000.0,269950.0


In [38]:
#Sort los Angeles data by date
df_LosAngeles = df_LosAngeles.sort_values(by='Date Occurred')

,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,...,Weapon Description,Status Code,Status Description,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,Location
254354,100610180,03/30/2010,01/01/2010,1200,6,Hollywood,635,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",0329,...,NaN,IC,Invest Cont,740.0,NaN,NaN,NaN,6700 WHITLEY TR,NaN,"(34.1103, -118.3333)"
319962,130509963,04/15/2013,01/01/2010,2300,5,Harbor,515,812,CRM AGNST CHLD (13 OR UNDER) (14-15 & SUSP 10 ...,NaN,...,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",AO,Adult Other,812.0,NaN,NaN,NaN,1600 BAYVIEW AV,NaN,"(33.795, -118.2711)"
293441,101225853,10/14/2010,01/01/2010,800,12,77th Street,1265,810,"SEX, UNLAWFUL",NaN,...,NaN,IC,Invest Cont,810.0,NaN,NaN,NaN,1400 W 83RD ST,NaN,"(33.9637, -118.3003)"
319964,130510649,05/30/2013,01/01/2010,1200,5,Harbor,525,354,THEFT OF IDENTITY,1202 1822,...,NaN,IC,Invest Cont,354.0,NaN,NaN,NaN,100 W D ST,NaN,"(33.7742, -118.262)"
309604,101905209,01/20/2010,01/01/2010,1,19,Mission,1966,354,THEFT OF IDENTITY,0377 1817,...,NaN,IC,Invest Cont,354.0,NaN,NaN,NaN,9100 VAN NUYS BL,NaN,"(34.2354, -118.4502)"
319965,130510915,06/06/2013,01/01/2010,600,5,Harbor,541,354,THEFT OF IDENTITY,0917 1822,...,NaN,IC,Invest Cont,354.0,NaN,NaN,NaN,1100 CAPITOL DR,NaN,"(33.7569, -118.3026)"
319966,130511194,06/13/2013,01/01/2010,800,5,Harbor,503,354,THEFT OF IDENTITY,0917 1822,...,NaN,IC,Invest Cont,354.0,NaN,NaN,NaN,1000 BAYVIEW LN,NaN,"(33.7915, -118.2939)"
319967,130511792,06/25/2013,01/01/2010,1,5,Harbor,514,354,THEFT OF IDENTITY,0931 1822,...,NaN,IC,Invest Cont,354.0,NaN,NaN,NaN,1600 WILMINGTON BL,NaN,"(33.7951, -118.2752)"
319969,130512975,07/26/2013,01/01/2010,1,5,Harbor,517,354,THEFT OF IDENTITY,1202 1822,...,NaN,IC,Invest Cont,354.0,NaN,NaN,NaN,1000 EUBANK AV,NaN,"(33.7831, -118.2552)"
138172,101604093,01/03/2010,01/01/2010,1800,16,Foothill,1601,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,12900 DEL SUR ST,NaN,"(34.2891, -118.4135)"
